In [36]:
!pip3 install ivon-opt
!pip install laplace-torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torchvision import datasets, transforms
from tqdm import tqdm

import numpy as np
import tqdm
import torch
from torchvision import datasets, transforms

from ivon import IVON
from torch.nn.utils import parameters_to_vector
import copy
from tqdm import tqdm

from laplace import Laplace
from laplace.curvature import AsdlGGN, AsdlEF
from torch.nn.utils import parameters_to_vector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00


## Not-MNIST dataset

In [40]:
!curl -o notMNIST_small.tar.gz http://yaroslavvb.com/upload/notMNIST/notMNIST_small.tar.gz
!curl -o notMNIST_large.tar.gz http://yaroslavvb.com/upload/notMNIST/notMNIST_large.tar.gz
!tar xzf notMNIST_small.tar.gz
!tar xzf notMNIST_large.tar.gz

import numpy, imageio, glob, sys, os, random

def get_labels_and_files(folder, number):
  # Make a list of lists of files for each label
  filelists = []
  for label in range(0,10):
    filelist = []
    filelists.append(filelist);
    dirname = os.path.join(folder, chr(ord('A') + label))
    for file in os.listdir(dirname):
      if (file.endswith('.png')):
        fullname = os.path.join(dirname, file)
        if (os.path.getsize(fullname) > 0):
          filelist.append(fullname)
        else:
          print('file ' + fullname + ' is empty')
    # sort each list of files so they start off in the same order
    # regardless of how the order the OS returns them in
    filelist.sort()

  # Take the specified number of items for each label and
  # build them into an array of (label, filename) pairs
  # Since we seeded the RNG, we should get the same sample each run
  labelsAndFiles = []
  for label in range(0,10):
    filelist = random.sample(filelists[label], number)
    for filename in filelist:
      labelsAndFiles.append((label, filename))

  return labelsAndFiles

def make_arrays(labelsAndFiles):
  images = []
  labels = []
  for i in range(0, len(labelsAndFiles)):

    # display progress, since this can take a while
    if (i % 100 == 0):
      sys.stdout.write("\r%d%% complete" % ((i * 100)/len(labelsAndFiles)))
      sys.stdout.flush()

    filename = labelsAndFiles[i][1]
    try:
      image = imageio.imread(filename)
      images.append(image)
      labels.append(labelsAndFiles[i][0])
    except:
      # If this happens we won't have the requested number
      print("\nCan't read image file " + filename)

  count = len(images)
  imagedata = numpy.zeros((count,28,28), dtype=numpy.uint8)
  labeldata = numpy.zeros(count, dtype=numpy.uint8)
  for i in range(0, len(images)):
    imagedata[i] = images[i]
    labeldata[i] = labels[i]
  print("\n")
  return imagedata, labeldata

def write_labeldata(labeldata, outputfile):
  header = numpy.array([0x0801, len(labeldata)], dtype='>i4')
  with open(outputfile, "wb") as f:
    f.write(header.tobytes())
    f.write(labeldata.tobytes())

def write_imagedata(imagedata, outputfile):
  header = numpy.array([0x0803, len(imagedata), 28, 28], dtype='>i4')
  with open(outputfile, "wb") as f:
    f.write(header.tobytes())
    f.write(imagedata.tobytes())
    
def convert(a,b,c,d):
  random.seed(42)
  labelsAndFiles = get_labels_and_files(a,b)
  print(len(labelsAndFiles))
  random.shuffle(labelsAndFiles)
  imagedata, labeldata = make_arrays(labelsAndFiles)
  write_labeldata(labeldata, c)
  write_imagedata(imagedata, d)


!mkdir -p not-mnist-data/MNIST/raw/
convert("notMNIST_small", 1000, "not-mnist-data/MNIST/raw/t10k-labels-idx1-ubyte", "not-mnist-data/MNIST/raw/t10k-images-idx3-ubyte")
convert("notMNIST_large", 6000, "not-mnist-data/MNIST/raw/train-labels-idx1-ubyte", "not-mnist-data/MNIST/raw/train-images-idx3-ubyte")
!gzip -k not-mnist-data/MNIST/raw/*ubyte
!rm -rf notMNIST_large/
!rm -rf notMNIST_small/
!rm -rf notMNIST_large.tar.gz
!rm -rf notMNIST_small.tar.gz

file notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is empty
file notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is empty
10000
2% complete

/tmp/ipykernel_34/3623616379.py:50: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


99% complete

file notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png is empty
file notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png is empty
file notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png is empty
file notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png is empty
60000
21% complete
Can't read image file notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png
99% complete



In [17]:
class State:
    def __init__(self, dim=(3,)):
        self.mean_x = torch.zeros(dim)
        self.mean_x2= torch.zeros(dim)
        self.idx = 0
        
    def mean(self):
        return self.mean_x
    
    def var(self):
        return self.mean_x2 - self.mean_x**2
    
    def update(self, x):
        new_idx = self.idx + 1
        self.mean_x = (self.mean_x * self.idx + x)/new_idx
        self.mean_x2 = (self.mean_x2 * self.idx + x**2)/new_idx
        self.idx = new_idx

In [37]:
TRANSFORMS = {
    'cifar10': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.1284, 0.1258, 0.1533])
        
    ]),
    'mnist': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.0433,)),
        transforms.Pad(2),
        transforms.Lambda(lambda x: torch.tile(x, (3,1,1)))

    ]),
    'nmnist':  transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4193,), (0.1850,)),
        transforms.Pad(2),
        transforms.Lambda(lambda x: torch.tile(x, (3,1,1)))

    ]),
    'fmnist': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.2860,), (0.1261,)),
        transforms.Pad(2),
        transforms.Lambda(lambda x: torch.tile(x, (3,1,1)))
    ]),
    'svhn': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4377, 0.4438, 0.4728], std=[0.1505, 0.1517, 0.1616]),
    ])
}

In [38]:
def get_vision_datasets():
    trainsets = []
    testsets = []
    
    trainsets += [datasets.MNIST('data/', train=True, download=False, transform=TRANSFORMS['mnist'])]
    testsets += [datasets.MNIST('data/', train=False, download=False, transform=TRANSFORMS['mnist'])]
    
    trainsets += [datasets.MNIST('not-mnist-data/', train=True, download=False, transform=TRANSFORMS['nmnist'])]
    testsets += [datasets.MNIST('not-mnist-data/', train=False, download=False, transform=TRANSFORMS['nmnist'])]
    
    trainsets += [datasets.FashionMNIST('fmnist/', train=True, download=False, transform=TRANSFORMS['fmnist'])]
    testsets += [datasets.FashionMNIST('fmnist/', train=False, download=False, transform=TRANSFORMS['fmnist'])]
    
    trainsets += [datasets.SVHN("svhn/", split="train", download=False, transform=TRANSFORMS['svhn'])]
    testsets += [datasets.SVHN("svhn/", split="test", download=False, transform=TRANSFORMS['svhn'])]
    
    trainsets += [datasets.CIFAR10('cifar10/', train=True, download=False, transform=TRANSFORMS['cifar10'])]
    testsets += [datasets.CIFAR10('cifar10/', train=False, download=False, transform=TRANSFORMS['cifar10'])]
    
    return trainsets, testsets

In [5]:
def get_disjoint_mnist_datasets(num_datasets):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transform)
    mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transform)

    train_labels = mnist_train.targets
    test_labels = mnist_test.targets

    class_per_dataset = 10 // num_datasets
    disjoint_datasets = []
    for i in range(num_datasets):
        classes = torch.arange(i * class_per_dataset, (i + 1) * class_per_dataset)
        train_indices = torch.nonzero(train_labels.unsqueeze(1) == classes, as_tuple=False)[:, 0]
        test_indices = torch.nonzero(test_labels.unsqueeze(1) == classes, as_tuple=False)[:, 0]
        train_subset = torch.utils.data.Subset(mnist_train, train_indices)
        test_subset = torch.utils.data.Subset(mnist_test, test_indices)
        disjoint_datasets.append((train_subset, test_subset))

    return disjoint_datasets

In [85]:
from torch.nn import functional as F

class MLP(torch.nn.Module):

    def __init__(self, input_size=28*28, output_size=10):
        super().__init__()
        self.l1 = torch.nn.Linear(input_size, 100)
        self.l2 = torch.nn.Linear(100, 100)
        self.l3 = torch.nn.Linear(100, 10)
        
    def forward(self, x):
        out = self.l1(x.view(-1, 28*28))
        out = self.l2(out.relu())
        out = self.l3(out.relu())
        return out
    
    
class LaNet(torch.nn.Module):
    def __init__(self, n_class=10, n_tasks=5):
        super().__init__()
        self.task_id = 0
        self.c1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.c2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=50*5*5, out_features=500),
            torch.nn.ReLU()
        )
        self.classifiers = torch.nn.ModuleList([torch.nn.Linear(in_features=500, out_features=n_class) for i in range(n_tasks)])
        
    def forward(self, x):
        out1 = self.c1(x)
        out2 = self.c2(out1)
        out3 = self.fc(out2.view(x.shape[0], -1))
        out = self.classifiers[self.task_id](out3)
        return out

In [136]:
class OnlineLearner():
    def __init__(self, cfg):
        self.cfg = cfg
        self.device = torch.device(cfg['device'])
        self.trainsets, self.testsets = get_vision_datasets()
        self.model = LaNet(n_class=10, n_tasks=5).to(self.device)
        self.current_parameters = copy.deepcopy(parameters_to_vector(self.model.parameters()).detach().clone())
        self.current_hessian = torch.zeros_like(self.current_parameters)
        self.validation_acc = []
        
    def evaluate(self, idx):
        acc = []
        for i, ds_val in enumerate(self.testsets[:idx+1]):
            dl = torch.utils.data.DataLoader(ds_val, batch_size=64)
            self.model.eval()
            correct, counts = 0, 0
            for x,y in dl:
                x = x.to(self.device)
                y = y.to(self.device)
                pred = self.model(x, idx=i).argmax(-1).view(-1)
                correct += (pred == y).sum().item()
                counts += x.shape[0]
            acc.append(correct/counts)
        print(f"Test accuracy: {acc}")
        return np.array(acc).mean()
    
    def ppd(self, idx):
        optimizer = IVON(self.model.parameters(), lr=self.cfg['lr'], ess=60000, hess_init=0.01)
        st = optimizer.state_dict()
        st['param_groups'][0]['hess'] = self.current_hessian 
        optimizer.load_state_dict(st)
        
        acc = []
        for i, ds_val in enumerate(self.testsets[:idx+1]):
            dl = torch.utils.data.DataLoader(ds_val, batch_size=512)
            self.model.eval()
            correct, counts = 0, 0
            for x,y in dl:
                x = x.to(self.device)
                y = y.to(self.device)
                sampled_probs = []
                for _ in range(100):
                    with optimizer.sampled_params():
                        sampled_logit = self.model(x, idx=i)
                        sampled_probs.append(F.softmax(sampled_logit, dim=1))
                        
                prob = torch.mean(torch.stack(sampled_probs), dim=0)

                pred = prob.argmax(-1).view(-1)
                correct += (pred == y).sum().item()
                counts += x.shape[0]
            acc.append(correct/counts)
        print(f"Test accuracy: {acc}")
        return np.array(acc).mean()
    
    def train(self, dataset, iters, idx):
        optimizer = IVON(self.model.parameters(), lr=self.cfg['lr'], ess=len(dataset), hess_init=0.1)
#         st = optimizer.state_dict()
#         st['param_groups'][0]['hess'] = self.current_hessian
#         optimizer.load_state_dict(st)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.5)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, eta_min=1e-6)
        self.model.train()
        for epoch in range(iters):
            data_loader = iter(torch.utils.data.DataLoader(dataset, batch_size=self.cfg['batch_size'], shuffle=True))
            for x, y in tqdm(data_loader):
                x = x.to(self.device)
                y = y.to(self.device)
                for _ in range(5):
                    with optimizer.sampled_params(train=True):
                        y_hat = self.model(x,idx=idx)
                        optimizer.zero_grad()
                        classifier_loss = torch.nn.functional.cross_entropy(input=y_hat, target=y, reduction='mean')
                        laplace_loss = (self.current_hessian * (parameters_to_vector(self.model.parameters()) - self.current_parameters)**2).sum()
                        loss = classifier_loss + self.cfg['lambda'] * laplace_loss
                        loss.backward()
                    
                optimizer.step()
            scheduler.step()
                #print(classifier_loss.item(),laplace_loss.item())
            #print(scheduler.get_last_lr())
            #self.evaluate(idx)
            
        
        # update currect map and hessian
        self.current_hessian = optimizer.state_dict()['param_groups'][0]['hess'].clone()
        self.current_parameters = copy.deepcopy(parameters_to_vector(self.model.parameters()).detach().clone())
            
    def train_all(self):
        for i in range(self.cfg['num_task']):
            print(f"Training task {i+1}...")
            self.train(self.trainsets[i], self.cfg['epoch'], idx=i)
            acc = self.evaluate(i)
            self.validation_acc.append(acc)
            print(f"Avg Test Accuracy: {acc: .3f}")
                
        #torch.save(torch.tensor(self.validation_acc), f"./plots/{self.cfg['train_mode']}.pt")

In [145]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.1,
    "epoch": 10,
    "train_mode": 'ivon',
    "lambda": 2.0,
    }

In [146]:
l = OnlineLearner(cfg)
l.train_all()

Training task 1...


100%|██████████| 235/235 [00:21<00:00, 11.09it/s]


Test accuracy: [0.9669]
Avg Test Accuracy:  0.967
Training task 2...


100%|██████████| 235/235 [00:21<00:00, 10.81it/s]


Test accuracy: [0.6901, 0.8716]
Avg Test Accuracy:  0.781
Training task 3...


...

Test accuracy: [0.7318, 0.8082, 0.7676]
Avg Test Accuracy:  0.769
Training task 4...


100%|██████████| 287/287 [00:23<00:00, 12.41it/s]


Test accuracy: [0.8335, 0.7371, 0.6894, 0.5894668100799016]
Avg Test Accuracy:  0.712
Training task 5...


100%|██████████| 196/196 [00:15<00:00, 12.90it/s]


Test accuracy: [0.799, 0.6772, 0.6695, 0.45540104486785493, 0.3425]
Avg Test Accuracy:  0.589


In [147]:
l2 = OnlineLearner(cfg)
l2.trainsets.reverse()
l2.testsets.reverse()
l2.train_all()

Training task 1...


100%|██████████| 196/196 [00:14<00:00, 13.16it/s]


Test accuracy: [0.5134]
Avg Test Accuracy:  0.513
Training task 2...


100%|██████████| 287/287 [00:23<00:00, 12.20it/s]


Test accuracy: [0.2311, 0.6958358942839582]
Avg Test Accuracy:  0.463
Training task 3...


...

Test accuracy: [0.2918, 0.6294176398279041, 0.8215]
Avg Test Accuracy:  0.581
Training task 4...


100%|██████████| 235/235 [00:21<00:00, 11.14it/s]


Test accuracy: [0.3201, 0.5234326982175783, 0.7974, 0.9095]
Avg Test Accuracy:  0.638
Training task 5...


100%|██████████| 235/235 [00:21<00:00, 10.95it/s]


Test accuracy: [0.3341, 0.5313460356484327, 0.7685, 0.898, 0.9777]
Avg Test Accuracy:  0.702


In [7]:
def getparam(vec, idx):
    addr = 652070
    x = addr + idx * 5010
    y = x+5010
    return torch.hstack([vec[:addr], vec[x:y]])

In [94]:
class OnlineLearner2():
    def __init__(self, cfg):
        self.cfg = cfg
        self.device = torch.device(cfg['device'])
        self.trainsets, self.testsets = get_vision_datasets()
        self.model = LaNet(n_class=10).to(self.device)
        self.la =  Laplace(self.model, 'classification',
                            subset_of_weights = 'all',
                            hessian_structure = cfg["hessian_structure"],
                            prior_mean = torch.zeros_like(parameters_to_vector(self.model.parameters())),
                            prior_precision = cfg["prior_prec_init"],
                            backend = AsdlGGN if cfg["approx_type"] == 'ggn' else AsdlEF)
        self.validation_acc = []
        
    def evaluate(self, idx):
        acc = []
        for i, ds_val in enumerate(self.testsets[:idx+1]):
            dl = torch.utils.data.DataLoader(ds_val, batch_size=64)
            self.model.eval()
            self.model.task_id = i
            correct, counts = 0, 0
            for x,y in dl:
                x = x.to(self.device)
                y = y.to(self.device)
                pred = self.model(x).argmax(-1).view(-1)
                correct += (pred == y).sum().item()
                counts += x.shape[0]
            acc.append(correct/counts)
        print(f"Test accuracy: {acc}")
        self.model.task_id = idx
        return np.array(acc).mean()
    
    def train(self, dataset, iters, idx):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.cfg['lr'], betas=(0.9, 0.999))
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.5)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, eta_min=1e-6)
        self.model.train()
        for epoch in range(iters):
            data_loader = iter(torch.utils.data.DataLoader(dataset, batch_size=self.cfg['batch_size'], shuffle=True))
            for x, y in tqdm(data_loader):
                x = x.to(self.device)
                y = y.to(self.device)
#                 for _ in range(5):
#                     with optimizer.sampled_params(train=True):
                y_hat = self.model(x)
                optimizer.zero_grad()
                classifier_loss = torch.nn.functional.cross_entropy(input=y_hat, target=y, reduction='mean')
                laplace_loss = - self.la.log_prob(getparam(parameters_to_vector(self.model.parameters()), idx)) / len(dataset) if idx !=0 else 0.0
                loss = classifier_loss + self.cfg['lambda'] * laplace_loss
                loss.backward()
                optimizer.step()
            scheduler.step()
                #print(classifier_loss.item(),laplace_loss.item())
            #print(scheduler.get_last_lr())
            #self.evaluate(idx)
            
        
#         # update currect map and hessian
#         self.current_hessian = optimizer.state_dict()['param_groups'][0]['hess'].clone()
#         self.current_parameters = copy.deepcopy(parameters_to_vector(self.model.parameters()).detach().clone())
            
    def train_all(self):
        for i in range(self.cfg['num_task']):
            print(f"Training task {i+1}...")
            self.model.task_id = i
            self.la.model.task_id = i
            self.train(self.trainsets[i], self.cfg['epoch'], idx=i)
            #self.la.mean = getparam(parameters_to_vector(self.model.parameters()), i).detach().clone()
            #self.la.H = getparam(self.la.)
            self.la.fit(torch.utils.data.DataLoader(self.trainsets[i], batch_size=self.cfg['batch_size'], shuffle=True), override=False)
            self.la.mean = getparam(parameters_to_vector(self.model.parameters()), i).detach().clone()
            acc = self.evaluate(i)
            self.validation_acc.append(acc)
            print(f"Avg Test Accuracy: {acc: .3f}")
                
        #torch.save(torch.tensor(self.validation_acc), f"./plots/{self.cfg['train_mode']}.pt")

In [71]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.001,
    "epoch": 1,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
        

In [76]:
l3 = OnlineLearner2(cfg)
l3.train_all()

Training task 1...


100%|██████████| 235/235 [00:16<00:00, 14.08it/s]


Test accuracy: [0.9866]
Avg Test Accuracy:  0.987
Training task 2...


100%|██████████| 235/235 [00:18<00:00, 12.88it/s]


Test accuracy: [0.9638, 0.9482]
Avg Test Accuracy:  0.956
Training task 3...


...

Test accuracy: [0.9547, 0.943, 0.8845]
Avg Test Accuracy:  0.927
Training task 4...


100%|██████████| 287/287 [00:19<00:00, 14.79it/s]


Test accuracy: [0.9138, 0.9112, 0.8442, 0.8610172095881992]
Avg Test Accuracy:  0.883
Training task 5...


100%|██████████| 196/196 [00:12<00:00, 15.32it/s]


Test accuracy: [0.7443, 0.8919, 0.8055, 0.78779963122311, 0.5701]
Avg Test Accuracy:  0.760


In [97]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.001,
    "epoch": 5,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ef", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
        

In [98]:
l3 = OnlineLearner2(cfg)
l3.train_all()

Training task 1...


100%|██████████| 235/235 [00:16<00:00, 13.88it/s]


Test accuracy: [0.9905]
Avg Test Accuracy:  0.991
Training task 2...


100%|██████████| 235/235 [00:18<00:00, 12.44it/s]


Test accuracy: [0.962, 0.9548]
Avg Test Accuracy:  0.958
Training task 3...


...

Test accuracy: [0.9042, 0.9381, 0.8997]
Avg Test Accuracy:  0.914
Training task 4...


100%|██████████| 287/287 [00:19<00:00, 14.57it/s]


Test accuracy: [0.9034, 0.9129, 0.8518, 0.8943992009834051]
Avg Test Accuracy:  0.891
Training task 5...


100%|██████████| 196/196 [00:13<00:00, 15.04it/s]


Test accuracy: [0.7538, 0.8834, 0.8145, 0.7992470805162877, 0.6012]
Avg Test Accuracy:  0.770


In [87]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.001,
    "epoch": 1,
    "hessian_structure": 'diag', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
l3 = OnlineLearner2(cfg)
l3.train_all()

Training task 1...


100%|██████████| 235/235 [00:17<00:00, 13.64it/s]


RuntimeError: The size of tensor a (677120) must match the size of tensor b (657080) at non-singleton dimension 0

In [93]:
l3.la.H

torch.Size([677120])

In [81]:
class LaNet(torch.nn.Module):
    def __init__(self, n_class=10):
        super().__init__()
        self.c1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.c2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=50*5*5, out_features=500),
            torch.nn.ReLU()
        )
        self.classifier =torch.nn.Linear(in_features=500, out_features=n_class)
    def forward(self, x):
        out1 = self.c1(x)
        out2 = self.c2(out1)
        out3 = self.fc(out2.view(x.shape[0], -1))
        out = self.classifier(out3)
        return out

In [82]:
class OnlineLearner2():
    def __init__(self, cfg):
        self.cfg = cfg
        self.device = torch.device(cfg['device'])
        self.trainsets, self.testsets = get_vision_datasets()
        self.model = LaNet(n_class=10).to(self.device)
        self.la =  Laplace(self.model, 'classification',
                            subset_of_weights = 'all',
                            hessian_structure = cfg["hessian_structure"],
                            prior_mean = torch.zeros_like(parameters_to_vector(self.model.parameters())),
                            prior_precision = cfg["prior_prec_init"],
                            backend = AsdlGGN if cfg["approx_type"] == 'ggn' else AsdlEF)
        self.validation_acc = []
        
    def evaluate(self, idx):
        acc = []
        for i, ds_val in enumerate(self.testsets[:idx+1]):
            dl = torch.utils.data.DataLoader(ds_val, batch_size=64)
            self.model.eval()
            correct, counts = 0, 0
            for x,y in dl:
                x = x.to(self.device)
                y = y.to(self.device)
                pred = self.model(x).argmax(-1).view(-1)
                correct += (pred == y).sum().item()
                counts += x.shape[0]
            acc.append(correct/counts)
        print(f"Test accuracy: {acc}")
        return np.array(acc).mean()
    
    def train(self, dataset, iters, idx):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.cfg['lr'], betas=(0.9, 0.999))
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.5)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, eta_min=1e-6)
        self.model.train()
        for epoch in range(iters):
            data_loader = iter(torch.utils.data.DataLoader(dataset, batch_size=self.cfg['batch_size'], shuffle=True))
            for x, y in tqdm(data_loader):
                x = x.to(self.device)
                y = y.to(self.device)
                y_hat = self.model(x)
                optimizer.zero_grad()
                classifier_loss = torch.nn.functional.cross_entropy(input=y_hat, target=y, reduction='mean')
                laplace_loss = - self.la.log_prob(parameters_to_vector(self.model.parameters())) / len(dataset) 
                loss = classifier_loss + self.cfg['lambda'] * laplace_loss
                loss.backward()
                optimizer.step()
            scheduler.step()

            
    def train_all(self):
        for i in range(self.cfg['num_task']):
            print(f"Training task {i+1}...")
            self.train(self.trainsets[i], self.cfg['epoch'], idx=i)
            self.la.fit(torch.utils.data.DataLoader(self.trainsets[i], batch_size=self.cfg['batch_size'], shuffle=True), override=False)
            acc = self.evaluate(i)
            self.validation_acc.append(acc)
            print(f"Avg Test Accuracy: {acc: .3f}")
                
        #torch.save(torch.tensor(self.validation_acc), f"./plots/{self.cfg['train_mode']}.pt")

In [83]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.001,
    "epoch": 1,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
l3 = OnlineLearner2(cfg)
l3.train_all()

Training task 1...


100%|██████████| 235/235 [00:17<00:00, 13.64it/s]


Test accuracy: [0.984]
Avg Test Accuracy:  0.984
Training task 2...


100%|██████████| 235/235 [00:17<00:00, 13.64it/s]


Test accuracy: [0.202, 0.9427]
Avg Test Accuracy:  0.572
Training task 3...


...

Test accuracy: [0.1724, 0.3644, 0.8767]
Avg Test Accuracy:  0.471
Training task 4...


100%|██████████| 287/287 [00:17<00:00, 16.28it/s]


Test accuracy: [0.5767, 0.1442, 0.2771, 0.8660110633066994]
Avg Test Accuracy:  0.466
Training task 5...


100%|██████████| 196/196 [00:11<00:00, 16.38it/s]


Test accuracy: [0.0985, 0.1452, 0.0545, 0.21008758451137063, 0.5259]
Avg Test Accuracy:  0.207


In [84]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.001,
    "epoch": 1,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
l3 = OnlineLearner2(cfg)
l3.train_all()

Training task 1...


100%|██████████| 235/235 [00:17<00:00, 13.66it/s]


Test accuracy: [0.9832]
Avg Test Accuracy:  0.983
Training task 2...


100%|██████████| 235/235 [00:18<00:00, 12.69it/s]


Test accuracy: [0.5272, 0.9471]
Avg Test Accuracy:  0.737
Training task 3...


...

Test accuracy: [0.3939, 0.8408, 0.8761]
Avg Test Accuracy:  0.704
Training task 4...


100%|██████████| 287/287 [00:19<00:00, 15.01it/s]


Test accuracy: [0.6195, 0.2984, 0.6211, 0.8495697602950215]
Avg Test Accuracy:  0.597
Training task 5...


100%|██████████| 196/196 [00:13<00:00, 15.08it/s]


Test accuracy: [0.2498, 0.4022, 0.4744, 0.47487707437000615, 0.5408]
Avg Test Accuracy:  0.428


New Experiments

In [39]:
from torch.nn import functional as F
   
class LaNet(torch.nn.Module):
    def __init__(self, n_class=10, n_tasks=5):
        super().__init__()
        self.last_layers = [torch.nn.Linear(in_features=500, out_features=n_class, bias=False) for i in range(n_tasks)]
        
        self.c1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=20, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.c2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=50*5*5, out_features=500),
            torch.nn.ReLU()
        )
        self.classifier = torch.nn.Linear(in_features=500, out_features=n_class, bias=False)
        self.switch_task(0)
    
    def switch_task(self, idx):
        assert idx < len(self.last_layers)
        self.classifier.weight.data = copy.deepcopy(self.last_layers[idx].weight.data)
        
    def update_task(self, idx):
        assert idx < len(self.last_layers)
        self.last_layers[idx].weight.data = copy.deepcopy(self.classifier.weight.data)
        
    def forward(self, x):
        out1 = self.c1(x)
        out2 = self.c2(out1)
        out3 = self.fc(out2.view(x.shape[0], -1))
        out = self.classifier(out3)
        return out

In [40]:

def get_hessian(model, dataset, cfg):
    la =  Laplace(model, 'classification',
                            subset_of_weights = 'all',
                            hessian_structure = cfg["hessian_structure"],
                            backend = AsdlGGN if cfg["approx_type"] == 'ggn' else AsdlEF)
    
    la.fit(torch.utils.data.DataLoader(dataset, batch_size=cfg['batch_size'], shuffle=True))#, progress_bar=True)
    return la.H

In [43]:
class OnlineLearner():
    def __init__(self, cfg):
        self.cfg = cfg
        self.device = torch.device(cfg['device'])
        self.trainsets, self.testsets = get_vision_datasets()
        self.model = LaNet(n_class=10).to(self.device)
        self.current_map = None
        self.current_hessian = None
        self.validation_acc = []
        self.beta = 1.0/2
        
    def log_prior(self):
        if  self.current_map != None:
            dim = torch.prod(torch.tensor(self.model.classifier.weight.shape)).item()
            delta = (parameters_to_vector(self.model.parameters()) - self.current_map)
            delta[-dim:] = 0.0  # ignoring classifier layer
            out  = 0.0
            if self.cfg['hessian_structure'] == 'kron':
#                 out = sum([delta @ self.current_hessian[i].bmm(delta, exponent=1) for i in range(len(self.current_hessian))])
                out = delta @ self.current_hessian.bmm(delta, exponent=1)
            if self.cfg['hessian_structure'] == 'diag':
                out = (delta**2 * self.current_hessian ).sum()#delta @ self.current_hessian.bmm(delta, exponent=1)
            return out
        return 0.0
    

    
    def train(self, dataset, iters):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.cfg['lr'], betas=(0.9, 0.999))
#        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.5)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, iters*len(dataset)//self.cfg['batch_size'], eta_min=1e-6)
        self.model.train()
        for epoch in range(iters):
            data_loader = iter(torch.utils.data.DataLoader(dataset, batch_size=self.cfg['batch_size'], shuffle=True))
            for x, y in tqdm(data_loader):
                x = x.to(self.device)
                y = y.to(self.device)
                y_hat = self.model(x)
                optimizer.zero_grad()
                likelihood_loss = torch.nn.functional.cross_entropy(input=y_hat, target=y, reduction='mean')   
                loss = likelihood_loss + self.beta * self.log_prior()
                loss.backward()
                optimizer.step()
                #scheduler.step()
                #print(classifier_loss.item(),laplace_loss.item())
            #print(scheduler.get_last_lr())
            #self.evaluate(idx)

    def evaluate(self, idx):
        acc = []
        for i, ds_val in enumerate(self.testsets[:idx+1]):
            dl = torch.utils.data.DataLoader(ds_val, batch_size=64)
            self.model.eval()
            self.model.switch_task(i)
            self.model.to(self.device)
            correct, counts = 0, 0
            for x,y in dl:
                x = x.to(self.device)
                y = y.to(self.device)
                pred = self.model(x).argmax(-1).view(-1)
                correct += (pred == y).sum().item()
                counts += x.shape[0]
            acc.append(correct/counts)
        print(f"Test accuracy: {acc}")
        return torch.tensor(acc)
    
    def train_all(self):
        for i in range(self.cfg['num_task']):
            print(f"Training task {i+1}...")
            self.model.switch_task(i)
            self.model.to(self.device)
            self.train(self.trainsets[i], self.cfg['epoch'])
            self.model.update_task(i)
            # laplace's approximation
            self.current_map = parameters_to_vector(self.model.parameters()).detach().clone()
            self.current_hessian  = get_hessian(self.model, self.trainsets[i], cfg)
        
            acc = self.evaluate(i)
            self.validation_acc.append(acc)
            print(f"Avg Test Accuracy: {acc.mean(): .3f}")
                
        #torch.save(torch.tensor(self.validation_acc), f"./plots/{self.cfg['train_mode']}.pt")

In [17]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.0001,
    "epoch": 10,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
        

In [18]:
accs = []
for i in range(5):
    l = OnlineLearner(cfg)
    l.train_all()
    accs.append(l.validation_acc)
torch.save(accs, f"{cfg['approx_type']}_{cfg['hessian_structure']}.pt")

Training task 1...


100%|██████████| 235/235 [00:17<00:00, 13.08it/s]


Test accuracy: [0.9908]
Avg Test Accuracy:  0.991
Training task 2...


100%|██████████| 235/235 [00:18<00:00, 12.88it/s]


Test accuracy: [0.9859, 0.9311]
Avg Test Accuracy:  0.959
Training task 3...


...

Test accuracy: [0.9836, 0.9331, 0.8038]
Avg Test Accuracy:  0.907
Training task 4...


100%|██████████| 287/287 [00:19<00:00, 14.82it/s]


Test accuracy: [0.9836, 0.9314, 0.7998, 0.6150891210817455]
Avg Test Accuracy:  0.832
Training task 5...


100%|██████████| 196/196 [00:12<00:00, 16.05it/s]


Test accuracy: [0.9803, 0.9294, 0.7699, 0.6104409956976029, 0.4167]
Avg Test Accuracy:  0.741


In [ ]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.0001,
    "epoch": 10,
    "hessian_structure": 'kron', # ['diag', 'kron', 'full']
    "approx_type": "ef", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
accs = []
for i in range(5):
    l = OnlineLearner(cfg)
    l.train_all()
    accs.append(l.validation_acc)
torch.save(accs, f"{cfg['approx_type']}_{cfg['hessian_structure']}.pt")

Training task 1...


100%|██████████| 235/235 [00:20<00:00, 11.58it/s]


Test accuracy: [0.991]
Avg Test Accuracy:  0.991
Training task 2...


100%|██████████| 235/235 [00:21<00:00, 10.91it/s]


Test accuracy: [0.9432, 0.9296]
Avg Test Accuracy:  0.936
Training task 3...


...

100%|██████████| 287/287 [00:22<00:00, 12.87it/s]


Test accuracy: [0.6928, 0.8941, 0.8023, 0.6952212661339889]
Avg Test Accuracy:  0.771
Training task 5...


100%|██████████| 196/196 [00:14<00:00, 13.39it/s]


Test accuracy: [0.6925, 0.8934, 0.8016, 0.6950676090964967, 0.3896]
Avg Test Accuracy:  0.694
Training task 1...


 30%|██▉       | 70/235 [00:05<00:13, 12.22it/s]

In [31]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.0001,
    "epoch": 10,
    "hessian_structure": 'diag', # ['diag', 'kron', 'full']
    "approx_type": "ef", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
accs = []
for i in range(5):
    l = OnlineLearner(cfg)
    l.train_all()
    accs.append(l.validation_acc)
torch.save(accs, f"{cfg['approx_type']}_{cfg['hessian_structure']}.pt")

Training task 1...


100%|██████████| 235/235 [00:17<00:00, 13.35it/s]


Test accuracy: [0.9918]
Avg Test Accuracy:  0.992
Training task 2...


100%|██████████| 235/235 [00:17<00:00, 13.24it/s]


Test accuracy: [0.9611, 0.9323]
Avg Test Accuracy:  0.947
Training task 3...


...

Test accuracy: [0.9678, 0.9286, 0.8071]
Avg Test Accuracy:  0.901
Training task 4...


100%|██████████| 287/287 [00:18<00:00, 15.85it/s]


Test accuracy: [0.9581, 0.9286, 0.7954, 0.6680623847572219]
Avg Test Accuracy:  0.838
Training task 5...


100%|██████████| 196/196 [00:11<00:00, 16.99it/s]


Test accuracy: [0.9682, 0.9205, 0.7901, 0.6697141979102643, 0.418]
Avg Test Accuracy:  0.753


In [30]:
cfg = {"device": 'cuda',
    "num_task": 5,
    "num_class": 10,
    "seed": 42,
    "batch_size": 256,
    "lr": 0.0001,
    "epoch": 10,
    "hessian_structure": 'diag', # ['diag', 'kron', 'full']
    "approx_type": "ggn", # ['ggn', 'ef'] 
    "prior_structure": "scaler", # ['all', 'layerwise', 'scalar']
    "prior_prec_init": 1e-3,
    "lambda": 1,
    }
accs = []
for i in range(5):
    l = OnlineLearner(cfg)
    l.train_all()
    accs.append(l.validation_acc)
torch.save(accs, f"{cfg['approx_type']}_{cfg['hessian_structure']}.pt")

Training task 1...


100%|██████████| 235/235 [00:16<00:00, 13.90it/s]


Test accuracy: [0.9888]
Avg Test Accuracy:  0.989
Training task 2...


100%|██████████| 235/235 [00:17<00:00, 13.27it/s]


Test accuracy: [0.9112, 0.9272]
Avg Test Accuracy:  0.919
Training task 3...


...

Test accuracy: [0.9561, 0.9247, 0.8137]
Avg Test Accuracy:  0.898
Training task 4...


100%|██████████| 287/287 [00:17<00:00, 16.07it/s]


Test accuracy: [0.8905, 0.9176, 0.8042, 0.6246542716656422]
Avg Test Accuracy:  0.809
Training task 5...


100%|██████████| 196/196 [00:12<00:00, 16.28it/s]


Test accuracy: [0.8279, 0.9111, 0.7946, 0.6157037492317148, 0.4105]
Avg Test Accuracy:  0.712


In [25]:
l.current_hessian.bmm(l.current_hessian)

RuntimeError: batch1 must be a 3D tensor

In [26]:
l.current_hessian

tensor([6.8687e+02, 4.8105e+02, 2.6890e+02,  ..., 2.5395e+03, 6.4277e-05,
        7.6483e+02], device='cuda:0')

In [2]:
!ls

cifar10  ef_diag.pt  ggn_diag.pt  not-mnist-data
data	 fmnist      ggn_kron.pt  svhn


In [32]:
data = torch.load("ef_diag.pt")
data_mean = [torch.vstack([i[j] for i in data]).mean(0) for j in range(5)]
data_mean

[tensor([0.9912]),
 tensor([0.9774, 0.9305]),
 tensor([0.9696, 0.9248, 0.8156]),
 tensor([0.8874, 0.9123, 0.7660, 0.6806]),
 tensor([0.8443, 0.9060, 0.7563, 0.6779, 0.4019])]

In [33]:
[i.mean() for i in data_mean]

[tensor(0.9912),
 tensor(0.9539),
 tensor(0.9033),
 tensor(0.8116),
 tensor(0.7173)]

In [34]:
def bwt(d):
    bwts = []
    for i in range(1,len(d)):
        tmp = 0
        for j in range(i):
            #print((d[i][j] , d[j][j]))
            tmp += (d[i][j] - d[j][j])
        bwts += [tmp/i]
    return bwts

In [35]:
bwt(data_mean)

[tensor(-0.0138), tensor(-0.0136), tensor(-0.0572), tensor(-0.0583)]